# Analyzing Environmental Justice: Integrating Census and Harmonized Landsat Sentinel-2 (HLS) NDVI Data with LLMs in Google Colab

## Overview
This exercise guides environmental and social scientists through the process of analyzing environmental justice issues by combining US Census data with NASA's Harmonized Landsat-Sentinel (HLS) satellite imagery using Large Language Models and rpompt engineering. You'll develop skills in integrating geospatial analysis and socioeconomic datasets using to identify hotspots of social vulnerability.

## Learning Objectives
- Learn the basics of Python and using Google Colab
- Integrate socioeconomic and remote sensing datasets
- Perform spatial analysis to identify areas of environmental concern
- Visualize and communicate environmental justice findings
- Become confident in using LLMs and prompt engineering to help with all your coding!!!

## Prerequisites
- Basic understanding of coding in some language (R, Matlab, etc)
- Basic understanding of data manipulation, data types, and LLMs
- A great attitude and the eye of the tiger!

## Part 1: Brief Introduction to Colab and Setting Up Your Environment and Working Directory

### Key Tasks
- Introduction to Google Colab and its features
- Install a suite of packages we will use in our workspace
- Set up our working directory
---



In [ ]:
# To mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install pandas geopandas numpy matplotlib seaborn scikit-learn rasterio earthpy rasterstats

####<font color="#FF69B4">PROMPT: "I would like to set my working directory for my notebook and check that it worked"</font>


In [ ]:
import os

# Set your working driectory
wd = '/content/drive/MyDrive/SCIPE/Workshops/llm_env_justice'
os.chdir(wd)

# Check that it worked
print(os.getcwd())

## Part 2: Working with Census Data to Create a Social Vulnerability Index (SVI)

In this section, you'll work with a subset of the 2022 Census data from the Baltimore, MD region and create a basic social vulnerability index.

### Key Tasks
- Read in a .geojson file (essentially a spatial dataframe like a shapefile)
- Learn to investigate variables
- Create and view a simple social vulnerability index (SVI)


---



####<font color="#FF69B4">PROMPT: "I want to import a .geojson file with census data in it named baltimore_region_acs_2022_epsg4269.geojson from my working directory and get some info about it. I would like the variable to be called gdf"</font>

In [ ]:
import geopandas as gpd

# Import .geojson file as census_df
census_loc = '/content/drive/MyDrive/SCIPE/Workshops/llm_env_justice/baltimore_region_acs_2022_epsg4269.geojson'
gdf = gpd.read_file(census_loc)

# Display the first few rows to verify
print(gdf.head())

# Look at the general info of the file (columns names, types, etc)
gdf.info()


####<font color="#FF69B4">PROMPT: "Can you calculate three more metrics that include the proportion of people in poverty, the proportion of renters, and the proportion of the population that is nonwhite and add to gdf"<font>

Specifically calulate them in the following way:

gdf['prop_poverty'] = (gdf['population_below_poverty'] / gdf['total_population'])

gdf['prop_renter'] = (gdf['renter_occupied_units'] / gdf['total_occupied_units'])

gdf['prop_nonwhite'] = 1 - (gdf['white_population'] / gdf['total_population'])
<font>

In [ ]:
# Calculate proportion metrics for poverty, renter, and nonwhite populations
gdf['prop_poverty'] = (gdf['population_below_poverty'] / gdf['total_population'])
gdf['prop_renter'] = (gdf['renter_occupied_units'] / gdf['total_occupied_units'])
gdf['prop_nonwhite'] = 1 - (gdf['white_population'] / gdf['total_population'])

####<font color="skyblue">CHALLENGE: Can you get the LLM to give you code to show only the first 10 rows of gdf and only for the new columns you just created?</font>

In [ ]:
# Select the first 10 rows and only the new columns
print(gdf[['prop_poverty', 'prop_renter', 'prop_nonwhite']].head(10))

####<font color="#FF69B4">PROMPT: "I would like to plot the three new columns side-by-side with viridis colormaps\"</font>

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 3 subplots side-by-side
fig, axes = plt.subplots(1, 3, figsize=(20, 10))

# Plot Poverty Rate
gdf.plot(column='prop_poverty',
         cmap='viridis',
         legend=True,
         legend_kwds={'label': 'Proportion Below Poverty'},
         ax=axes[0])
axes[0].set_title('Proportion Below Poverty by Census Tract')
axes[0].set_axis_off() # Turn off axes for a cleaner map

# Plot Renter Proportion
gdf.plot(column='prop_renter',
         cmap='viridis',
         legend=True,
         legend_kwds={'label': 'Proportion Renter Occupied'},
         ax=axes[1])
axes[1].set_title('Proportion Renter Occupied by Census Tract')
axes[1].set_axis_off() # Turn off axes for a cleaner map

# Plot Proportion Nonwhite
gdf.plot(column='prop_nonwhite',
         cmap='viridis',
         legend=True,
         legend_kwds={'label': 'Proportion Nonwhite'},
         ax=axes[2])
axes[2].set_title('Proportion Nonwhite by Census Tract')
axes[2].set_axis_off() # Turn off axes for a cleaner map

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

####<font color="skyblue">CHALLENGE: Is there anything you would change about the way the plots look (e.g., colorbar size, font size, colormap, etc)? Ask the LLM and customize away! NOTE: If you get errors you can hit the "Explain Error" button in the lower left after the error and feed it right back into Gemini who will then try to correct your code!</font>

####<font color="#FF69B4">PROMPT: "I would like to make the colormaps "turbo", reduce the size of the colorbars and make the font slightly larger"</font>

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 3 subplots side-by-side
fig, axes = plt.subplots(1, 3, figsize=(20, 10))

# Define common legend properties (removed 'size')
legend_properties = {} # Adjust size as needed

# Plot Poverty Rate
# Store the Axes object returned by plot
ax1 = gdf.plot(column='prop_poverty',
         cmap='turbo',  # Changed colormap
         legend=True,
         legend_kwds={'label': 'Proportion Below Poverty', **legend_properties, 'shrink': 0.5}, # Adjusted legend properties and shrink
         ax=axes[0])
axes[0].set_title('Proportion Below Poverty by Census Tract', fontsize=14) # Adjusted title font size
axes[0].set_axis_off() # Turn off axes for a cleaner map

# Plot Renter Proportion
# Store the Axes object returned by plot
ax2 = gdf.plot(column='prop_renter',
         cmap='turbo',  # Changed colormap
         legend=True,
         legend_kwds={'label': 'Proportion Renter Occupied', **legend_properties, 'shrink': 0.5}, # Adjusted legend properties and shrink
         ax=axes[1])
axes[1].set_title('Proportion Renter Occupied by Census Tract', fontsize=14) # Adjusted title font size
axes[1].set_axis_off() # Turn off axes for a cleaner map

# Plot Proportion Nonwhite
# Store the Axes object returned by plot
ax3 = gdf.plot(column='prop_nonwhite',
         cmap='turbo',  # Changed colormap
         legend=True,
         legend_kwds={'label': 'Proportion Nonwhite', **legend_properties, 'shrink': 0.5}, # Adjusted legend properties and shrink
         ax=axes[2])
axes[2].set_title('Proportion Nonwhite by Census Tract', fontsize=14) # Adjusted title font size
axes[2].set_axis_off() # Turn off axes for a cleaner map

# If you need to adjust colorbar font size after plotting:
# This requires accessing the colorbar object, which is not directly returned by gdf.plot.
# A more complex approach involving creating an explicit colorbar axis might be needed
# if fine-grained control over colorbar font size is essential with gdf.plot legend=True.
# For simplicity, removing the 'size' keyword is the direct fix for the Type Error.


# Adjust layout and display the plot
plt.tight_layout()
plt.show()

####<font color="#FF69B4">PROMPT: "I would like to create a Social Vulnerability Index (SVI) from my three new metrics and plot it"</font>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Create a Social Vulnerability Index (SVI)
# Normalize and combine poverty, income, and housing metrics
# Select variables for SVI
svi_vars = ['prop_poverty', 'prop_renter', 'prop_nonwhite'] # Use the proportion columns
svi_df = gdf[svi_vars].copy()

# Handle missing values
svi_df.fillna(svi_df.mean(), inplace=True)

# Normalize the variables (0-1 scale)
scaler = MinMaxScaler()
svi_normalized = scaler.fit_transform(svi_df)
svi_normalized_df = pd.DataFrame(svi_normalized, columns=svi_vars, index=svi_df.index)

# Calculate SVI (simple average of normalized variables)
gdf['social_vulnerability_index'] = svi_normalized_df.mean(axis=1)

# Visualize social vulnerability
fig, ax = plt.subplots(1, figsize=(15, 10))
gdf.plot(column='social_vulnerability_index',
                cmap='viridis',
                legend=True,
                legend_kwds={'label': 'Social Vulnerability Index'},
                ax=ax)
plt.title('Social Vulnerability by Census Tract')
plt.savefig('social_vulnerability_map.png', dpi=300, bbox_inches='tight')
plt.show()

####<font color="#FF69B4">PROMPT: "I am curious what the MinMaxScaler did to the data? Can you compare before and after it was applied?"</font>

In [ ]:
# Display the head of the data BEFORE MinMaxScaler
print("Data BEFORE MinMaxScaler:")
print(svi_df.head())

print("\n" + "="*30 + "\n") # Separator for clarity

# Display the head of the data AFTER MinMaxScaler
print("Data AFTER MinMaxScaler:")
print(svi_normalized_df.head())

####<font color="#FF69B4">PROMPT: "I would like to see a histogram of the SVI data"</font>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Import pandas in case it wasn't imported earlier in this block

# Create a histogram of the social_vulnerability_index column
plt.figure(figsize=(10, 6)) # Set the size of the figure
sns.histplot(gdf['social_vulnerability_index'].dropna(), # Select the column and drop any potential missing values
             kde=True, # Add a Kernel Density Estimate line for a smoother distribution shape
             bins=50) # Increase the number of bins for more detail

plt.title('Distribution of Social Vulnerability Index') # Set the title of the plot
plt.xlabel('Social Vulnerability Index') # Set the x-axis label
plt.ylabel('Frequency') # Set the y-axis label
plt.grid(axis='y', alpha=0.75) # Add a grid for easier reading
plt.show() # Display the plot

## Part 3: Integrating Harmonized Landsat Sentinel-2 NDVI Data with Census Data to Illuminate Patterns of Social Vulnerability

In this section, you will work with a summer 2022 Normalized Difference Vegetation Index (NDVI) composite dataset derived from Harmonized Landsat Sentinel-2 [(HLS info)](https://hls.gsfc.nasa.gov/) and integrate it with the 2022 Census data for deeper exploration in the Baltimore, MD region.

### Key Tasks
- Import and view a geotiff raster file
- Extract NDVI statistics for each census tract and merge data into new colums
- Calculate a Green Space Inequity Index and view it
- Create a list of high priority environmental justice areas

### NDVI Primer
NDVI (Normalized Difference Vegetation Index) is a widely used remote sensing index that quantifies vegetation health and density using satellite or aerial imagery. It is calculated from the visible and near-infrared light reflected by vegetation.
<br>
<br>

📊 NDVI Formula:
$$
\text{NDVI} = \frac{(NIR - RED)}{(NIR + RED)}
$$

- NIR = Reflectance in the near-infrared spectrum (plants strongly reflect this light)

- RED = Reflectance in the red spectrum (plants absorb this light for photosynthesis)
<br>

🌱 NDVI Values:
NDVI values range from -1 to +1, and they indicate:

| NDVI Value    | Interpretation                           |
| ------------- | ---------------------------------------- |
| **< 0**       | Water, clouds, snow, or bare soil        |
| **0.1 – 0.2** | Sparse vegetation (e.g., shrubs)         |
| **0.2 – 0.5** | Moderate vegetation (e.g., grassland)    |
| **0.5 – 0.9** | Dense, healthy vegetation (e.g., forest) |
<br>
<br>

🛰️ Why is NDVI Useful?
- Environmental Monitoring: Assess drought, deforestation, and land degradation.
- Agriculture: Monitor crop health and stress.
- Urban Planning: Evaluate green space distribution.
- Climate Studies: Understand carbon sinks and land surface processes.

---

####<font color="#FF69B4">PROMPT: "I want to load my NDVI geotiff now and plot the data"</font>

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Path to the NDVI geotiff
ndvi_path = '/content/drive/MyDrive/SCIPE/Workshops/llm_env_justice/baltimore_region_HLSS30_mean_summer_2022_NDVI_epsg4269_clip.tif'

# Open the GeoTIFF
try:
    with rasterio.open(ndvi_path) as src:
        ndvi = src.read(1)  # Read the first band
        ndvi_transform = src.transform # Get the affine transform
        ndvi_crs = src.crs # Get the CRS

    print("GeoTIFF opened successfully.")
    print(f"GeoTIFF CRS: {ndvi_crs}")
    # You might want to compare this to your gdf CRS:
    # print(f"GeoDataFrame CRS: {gdf.crs}") # Assuming gdf is already loaded

    # Plot the NDVI data
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    im = ax.imshow(ndvi, cmap='YlGn', vmin=-1, vmax=1) # Set value range from -1 to 1
    fig.colorbar(im, ax=ax, label='NDVI')
    plt.title('Mean Summer 2022 NDVI - Baltimore Region')
    plt.xlabel('Column #') # Pixel column index
    plt.ylabel('Row #') # Pixel row index
    plt.show()

except rasterio.errors.RasterioIOError as e:
    print(f"Error opening GeoTIFF: {e}")
    print("Please check the file path and ensure the file exists.")
except Exception as e:
    print(f"An error occurred during plotting: {e}")

####<font color="#FF69B4">PROMPT: "I want to extract statistics from the ndvi dataset for each tract in my census data and add the data as new columns into my census dataset. Also make sure that the nodata is set for all values equal to -9999 so they aren't being considered"</font>

In [ ]:
# Extract zonal NDVI statistics for each census tract in gdf

# Import rasterstats
import rasterstats
import pandas as pd # Import pandas if not already available in this block

# Stats to calculate
stats_to_calculate = ['mean', 'min', 'max', 'median', 'count'] # Example stats

# Extract zonal statistics
zonal_statistics = rasterstats.zonal_stats(
    gdf.geometry,       # Geometries from the GeoDataFrame
    ndvi,       # Raster data (NumPy array)
    affine=ndvi_transform, # Affine transform of the raster
    stats=stats_to_calculate, # List of statistics to calculate
    nodata=-9999 # Specify the no-data value
    )

print(f"Calculated zonal statistics for {len(zonal_statistics)} features.")

# Convert the list of dictionaries to a pandas DataFrame
stats_df = pd.DataFrame(zonal_statistics)

# Append the new columns to the original GeoDataFrame
# Assuming the order of results matches the order of geometries in gdf
gdf = gdf.join(stats_df)

# Rename the new columns for clarity (optional but recommended)
new_col_names = {stat: f'ndvi_{stat}' for stat in stats_to_calculate}
gdf = gdf.rename(columns=new_col_names)

# Display the first few rows to see the new columns
print(gdf.head())

####<font color="#FF69B4">PROMPT: "I would like to create a new green space inequity index that uses the census data and the ndvi data and plot the data on a scatter plot"</font>

In [ ]:
# Calculate Green Space Inequity Index
# Higher index means less green space relative to social vulnerability
gdf['green_space_inequity'] = (1 - gdf['ndvi_mean']) * gdf['social_vulnerability_index']

# Visualize the relationship between social vulnerability and NDVI
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(
    gdf['social_vulnerability_index'],
    gdf['ndvi_mean'],
    c=gdf['green_space_inequity'],
    cmap='viridis',
    alpha=0.7,
    s=50
)
plt.colorbar(scatter, label='Green Space Inequity Index')
plt.xlabel('Social Vulnerability Index')
plt.ylabel('Mean NDVI (Vegetation Index)')
plt.title('Relationship Between Social Vulnerability and Vegetation Cover')
plt.tight_layout()
# plt.savefig('vulnerability_vs_ndvi.png', dpi=300)
plt.show()

# Map the Green Space Inequity Index
fig, ax = plt.subplots(1, figsize=(15, 10))
gdf.plot(
    column='green_space_inequity',
    cmap='RdYlGn_r',  # Reversed colormap (red = high inequity)
    legend=True,
    legend_kwds={'label': 'Green Space Inequity Index'},
    ax=ax
)
plt.title('Green Space Inequity by Census Tract')
plt.savefig('green_space_inequity_map.png', dpi=300, bbox_inches='tight')
plt.show()

####<font color="#FF69B4">PROMPT: "I would like to plot maps of SVI and the Green Space Inequity Index next to each other to compare"</font>:

In [ ]:
import matplotlib.pyplot as plt

# Choose a colormap to use for both plots
common_cmap = 'viridis' # Or use 'RdYlGn_r' or any other colormap

# Create a figure with 2 subplots side-by-side
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot Social Vulnerability Index
gdf.plot(column='social_vulnerability_index',
         cmap=common_cmap, # Use the chosen common colormap
         legend=True,
         legend_kwds={'label': 'Social Vulnerability Index'},
         ax=axes[0])
axes[0].set_title('Social Vulnerability by Census Tract')
axes[0].set_axis_off()

# Plot Green Space Inequity Index
gdf.plot(column='green_space_inequity',
         cmap=common_cmap,  # Use the chosen common colormap
         legend=True,
         legend_kwds={'label': 'Green Space Inequity Index'},
         ax=axes[1])
axes[1].set_title('Green Space Inequity by Census Tract')
axes[1].set_axis_off()

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

####<font color="#FF69B4">PROMPT: "Can we run and view a correlation matrix for the variables that go into the green space inequity index?"</font>


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Select the columns for the correlation matrix
# These are the three proportion metrics and the mean NDVI
correlation_vars = ['prop_poverty', 'prop_renter', 'prop_nonwhite', 'ndvi_mean']

# Create a subset DataFrame with only these columns
corr_df = gdf[correlation_vars].copy()

# Calculate the correlation matrix
# .corr() method calculates the pairwise correlation of columns, excluding NA/null values
correlation_matrix = corr_df.corr()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

# Optional: Visualize the correlation matrix using a heatmap
plt.figure(figsize=(8, 6)) # Set figure size for the heatmap
sns.heatmap(correlation_matrix,
            annot=True,     # Annotate cells with the correlation values
            cmap='coolwarm', # Colormap (e.g., coolwarm, viridis, plasma)
            fmt=".2f",      # Format the annotations to 2 decimal places
            linewidths=.5)  # Add lines between cells for clarity

plt.title('Correlation Matrix of Proportion Metrics and Mean NDVI') # Set the title
plt.show() # Display the plot

####<font color="#FF69B4">PROMPT: "Can we make a plot showing census tracts with the 10 highest environmental justice priority scores based on the green space inequity index?"</font>:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Calculate a priority score based on the green space inequity index.
# You can adjust this formula based on how you want to weight inequity.
# For simplicity, we'll just use the green_space_inequity directly for ranking.
# If you wanted to combine other factors, you would do it here.
gdf['priority_score'] = gdf['green_space_inequity']

# Handle any potential NaN values in 'priority_score' before sorting
gdf_sorted = gdf.dropna(subset=['priority_score']).sort_values(by='priority_score', ascending=False)

# Select the top 10 high-priority tracts
top_10_tracts = gdf_sorted.head(10).copy()

# Print the top 10 tracts and their scores
print("Top 10 High-Priority Census Tracts:")
print(top_10_tracts[['tract_code', 'priority_score', 'social_vulnerability_index', 'ndvi_mean']]) # Display relevant info

# Create a base map of all census tracts
fig, ax = plt.subplots(1, figsize=(15, 10))
gdf.plot(ax=ax, color='lightgray', edgecolor='white', linewidth=0.5)

# Plot the top 10 high-priority tracts on top
top_10_tracts.plot(
    column='priority_score',
    cmap='OrRd',  # Use a color map to show score differences among the top 10
    legend=True,
    legend_kwds={'label': 'Priority Score'},
    ax=ax,
    edgecolor='black' # Add an edge color to make them stand out
)

# Add titles and turn off axes
plt.title('Census Tracts with Top 10 Highest Environmental Justice Priority Scores')
ax.set_axis_off()

# Show the plot
plt.show()

## Part 4: Go Wild with LLMs!!

### Key Tasks
- Have fun exploring the data further in anyway you see fit using LLMs as your guide!
- I have provided a clustering example prompt with the code available in the cheat sheet.
- Test your new Colab, Python, and LLM skills!
<br>
<br>

### Suggestions for exploring the data with LLMs
- Ask LLM to convert parts or all of this code into R, Matlab, etc
- Run principal component analysis on the data
- Try Google Colab with some of your own data
- Try to build a script to download data from an API like the Census Data API (you will likely need register to get an API key) [Census API User Guide](https://www.census.gov/data/developers/guidance/api-user-guide.html)



---

####<font color="#FF69B4">PROMPT: "I would like to run a cluster analysis on the variables that went into the SVI and ndvi_median"</font>

In [ ]:
# Cluster analysis to identify similar environmental justice patterns
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler # Make sure MinMaxScaler is imported

# Prepare data for clustering
# Select the SVI variables ('renter_rate', 'poverty_rate', 'pct_nonwhite') and 'ndvi_median'
cluster_data = gdf[['ndvi_median', 'prop_poverty', 'prop_renter', 'prop_nonwhite']].copy() # Corrected column names to use prop_*
cluster_data = cluster_data.dropna()  # Remove rows with missing values

# Normalize the data
scaler = MinMaxScaler() # Ensure scaler is defined (it was defined earlier, but good to be explicit here)
cluster_data_scaled = scaler.fit_transform(cluster_data)
cluster_data_scaled_df = pd.DataFrame(cluster_data_scaled, columns=cluster_data.columns, index=cluster_data.index) # Convert back to DataFrame to keep index

# Determine optimal number of clusters using the elbow method
inertia = []
k_range = range(1, 11) # Check k from 1 to 10
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # Added n_init for KMeans robustness
    kmeans.fit(cluster_data_scaled_df)
    inertia.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, 'o-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (Within-cluster sum of squares)')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()


In [ ]:
# --- Choose an optimal number of clusters based on the elbow curve ---
# After reviewing the elbow curve, you'll select a value for k.
# For example, if the elbow seems to be at k=4:
# k = 4 # Replace with your chosen optimal k
# Let's choose k=4 as an example to continue

k = 4 # Example: Based on a hypothetical elbow at k=4

# Run KMeans with the chosen number of clusters
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(cluster_data_scaled_df) # Fit and predict on the scaled data

# Add cluster information to the original GeoDataFrame
# We need to add the cluster assignments back to the original gdf based on the index
gdf['cluster'] = None # Initialize the column

# Make sure the indices align before joining. If not, re-index the cluster assignments.
# A safe way is to create a Series from the cluster assignments with the same index
cluster_series = pd.Series(clusters, index=cluster_data_scaled_df.index, name='cluster')

# Assign the cluster values back to the original gdf based on the index
gdf.loc[cluster_data_scaled_df.index, 'cluster'] = cluster_series

# Convert the cluster column to a categorical type for better plotting
gdf['cluster'] = gdf['cluster'].astype('category')


# Visualize the clusters on the map
fig, ax = plt.subplots(1, figsize=(15, 10))
# Ensure the 'cluster' column is treated as categorical
gdf.plot(
    column='cluster',
    cmap='tab10', # tab10 is a good categorical colormap
    categorical=True,
    legend=True,
    ax=ax
)
plt.title('Environmental Justice Pattern Clusters')
ax.set_axis_off()
plt.show()


In [ ]:
# Analyze cluster characteristics using the SCALED data
# Calculate the mean of the SCALED variables within each cluster

# Ensure 'cluster' column exists in the scaled DataFrame.
# Since we added 'cluster' to the original gdf based on the scaled_df index,
# we can join the cluster column back to the scaled df before grouping.
cluster_data_scaled_df_with_clusters = cluster_data_scaled_df.copy()

# Join the cluster assignments to the scaled DataFrame (using the previously created series)
cluster_data_scaled_df_with_clusters['cluster'] = cluster_series

# Now group by the cluster column
cluster_profiles_scaled = cluster_data_scaled_df_with_clusters.groupby('cluster')[
    ['ndvi_median', 'prop_poverty', 'prop_renter', 'prop_nonwhite'] # Corrected column names
].mean()

print("\nCluster Profiles (Mean SCALED values for each variable by cluster):")
print(cluster_profiles_scaled)

# Visualize cluster profiles (using scaled means)
cluster_profiles_scaled.plot(
    kind='bar',
    figsize=(12, 8),
    colormap='viridis' # Or another suitable colormap
)
plt.title('Average Characteristics of Environmental Justice Clusters (Scaled Data)')
plt.ylabel('Average Scaled Value')
plt.xticks(rotation=45, ha='right') # Rotate labels for readability
plt.tight_layout()
plt.show()

####<font color="#FF69B4">PROMPT: "Can you help me interpret these cluster profiles in a descriptive way?"

HINT: Try copying the table values printed out above into Gemini and asking to interpret the data</font>

####<font color="#FF69B4">PROMPT: "Can you plot cluster vs income?"</font>

---



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Ensure pandas is imported

# Make sure the 'cluster' column is created and populated in gdf
# This should have happened in the previous clustering steps.
# If not, you would need to re-run those steps.

# Ensure 'median_household_income' column exists and is numeric
if 'median_household_income' not in gdf.columns:
    print("Error: 'median_household_income' column not found in gdf. Please check your data.")
elif not pd.api.types.is_numeric_dtype(gdf['median_household_income']):
     print("Error: 'median_household_income' column is not numeric. Please check data types.")
else:
    # Create a box plot to show the distribution of median household income for each cluster
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='cluster', y='median_household_income', data=gdf.dropna(subset=['cluster', 'median_household_income'])) # Drop rows with missing cluster or income

    plt.title('Median Household Income Distribution by Environmental Justice Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Median Household Income')
    plt.grid(axis='y', alpha=0.75)
    plt.show()

    # Optional: Calculate and print the mean median household income for each cluster
    mean_household_income_by_cluster = gdf.groupby('cluster')['median_household_income'].mean()
    print("\nMean Median Household Income by Cluster:")
    print(mean_household_income_by_cluster)


####<font color="#FF69B4">PROMPT: "I would like to plot ndvi vs income with points colored by the Green Space Inequity Index"</font>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Ensure pandas is imported

# Ensure the required columns exist and are numeric
required_cols = ['median_household_income', 'ndvi_mean', 'green_space_inequity']
if not all(col in gdf.columns for col in required_cols):
    print(f"Error: One or more required columns {required_cols} not found in gdf. Please check your data.")
elif not all(pd.api.types.is_numeric_dtype(gdf[col]) for col in required_cols):
    print("Error: One or more of the required columns are not numeric. Please check data types.")
else:
    # Create the scatter plot colored by Green Space Inequity
    plt.figure(figsize=(10, 6))

    scatter = plt.scatter(
        gdf['median_household_income'],  # X-axis: Income
        gdf['ndvi_mean'],              # Y-axis: Mean NDVI
        c=gdf['green_space_inequity'],  # Color: Green Space Inequity Index
        cmap='RdYlGn_r',               # Colormap (Red=High Inequity, Green=Low Inequity)
        alpha=0.7,                     # Adjust transparency
        s=50                           # Adjust point size
    )

    # Add a color bar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Green Space Inequity Index')

    plt.title('Mean NDVI vs. Median Household Income, Colored by Green Space Inequity')
    plt.xlabel('Median Household Income')
    plt.ylabel('Mean NDVI (Vegetation Index)')
    plt.grid(True, linestyle='--', alpha=0.6)

    # Optional: Add annotations for specific points if needed

    plt.show()


## Part 5: Show and Tell: Creating a Dashboard for Environmental Justice Data Exploration

In this final section, I'll show you a data dashboard that I created wholly by interacting with LLMs and how powerful Colab and LLMs can be for creating interactive visualizations to share with stakeholders.

#### YOU CAN ONLY RUN THIS IF YOU HAVE AN NGROK AUTHENITICATION TOKEN SO I WILL JUST SHOW YOU A LIVE DEMO


---

In [ ]:
# Install required packages
!pip install dash plotly pyngrok pyproj

import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pyproj
from shapely.geometry import Point
import time
import threading

from pyngrok import ngrok

TOKEN = "YOUR NGROK TOKEN HERE"

high_priority = top_10_tracts.copy()

# Add your ngrok authentication token here
try:
    ngrok.set_auth_token(TOKEN)
    print("ngrok authtoken set successfully.")
except Exception as e:
    print(f"Error setting ngrok authtoken: {e}")

# Convert GeoDataFrame to GeoJSON for Plotly
try:
    census_geojson = gdf.__geo_interface__
    print("GeoDataFrame converted to GeoJSON interface.")
except Exception as e:
    print(f"Error converting GeoDataFrame to GeoJSON interface: {e}")
    census_geojson = {}

# Calculate map center using projected centroid
projected_crs_epsg = 26918 # NAD83 / Maryland State Plane (Feet)

print(f"Original GeoDataFrame CRS: {gdf.crs}")
print(f"Reprojecting GeoDataFrame to projected CRS: EPSG:{projected_crs_epsg} for centroid calculations.")

try:
    if not gdf.crs or not gdf.crs.is_geographic:
         print("Warning: Original gdf is not in a geographic CRS. Attempting to reproject to geographic (EPSG:4269) before projecting for centroids.")
         gdf = gdf.to_crs(epsg=4269)
         print(f"gdf reprojected to geographic: {gdf.crs}")

    gdf_projected = gdf.to_crs(epsg=projected_crs_epsg)
    print(f"GeoDataFrame reprojected to: {gdf_projected.crs}")

    valid_geometries_projected = gdf_projected.geometry.is_valid & ~gdf_projected.geometry.is_empty
    if valid_geometries_projected.any():
        valid_gdf_projected = gdf_projected[valid_geometries_projected].copy()
        mean_x_projected = valid_gdf_projected.geometry.centroid.x.mean()
        mean_y_projected = valid_gdf_projected.geometry.centroid.y.mean()
        mean_centroid_projected = Point(mean_x_projected, mean_y_projected)

        transformer = pyproj.Transformer.from_crs(f"EPSG:{projected_crs_epsg}", f"EPSG:{gdf.crs.to_epsg()}", always_xy=True)
        center_lon_geographic, center_lat_geographic = transformer.transform(mean_centroid_projected.x, mean_centroid_projected.y)
        map_center = {"lat": center_lat_geographic, "lon": center_lon_geographic}
        print(f"Calculated map center (geographic): {map_center}")
    else:
        print("Error: No valid projected geometries found to calculate mean centroid.")
        raise ValueError("No valid geometries for centroid calculation.")

except Exception as e:
    print(f"Error during reprojection or centroid calculation: {e}")
    print("Using default geographic center.")
    map_center = {"lat": 39.2904, "lon": -76.6122} # Approximate center of Baltimore
    print(f"Using default map center: {map_center}")

# Reset index to ensure we have clean integer indices
gdf = gdf.reset_index(drop=True)
print(f"GeoDataFrame shape after reset: {gdf.shape}")
print(f"GeoDataFrame index: {gdf.index[:5].tolist()}")

# Create initial Social Vulnerability map figure
svi_fig = px.choropleth_mapbox(
    gdf,
    geojson=census_geojson,
    locations=gdf.index,  # Use integer index
    color='social_vulnerability_index',
    color_continuous_scale='Viridis',
    range_color=[0, 1],
    mapbox_style='carto-positron',
    zoom=9,
    center=map_center,
    opacity=0.7,
    hover_name='tract_code',  # Changed from 'NAME' to 'tract_code'
    hover_data={
        'ndvi_mean': ':.3f',
        'social_vulnerability_index': ':.3f',
        'prop_poverty': ':.3f',  # Changed from 'poverty_rate'
        'prop_nonwhite': ':.3f',  # Changed from 'pct_nonwhite'
        'priority_score': ':.3f'
    },
    labels={
        'ndvi_mean': 'Vegetation Index',
        'social_vulnerability_index': 'Social Vulnerability',
        'prop_poverty': 'Poverty Rate',  # Updated label
        'prop_nonwhite': 'Non-white Population',  # Updated label
        'priority_score': 'Priority Score',
        'tract_code': 'Tract Code'  # Added label
    },
    title='Social Vulnerability by Census Tract'
)

svi_fig.update_layout(
    mapbox_layers=[],
    uirevision='constant'
)

# Create scatter plot - KEY: Make sure this uses the same DataFrame in same order
scatter_fig = px.scatter(
    gdf,
    x='social_vulnerability_index',
    y='ndvi_mean',
    color='priority_score',
    size='total_population',
    hover_name='tract_code',  # Changed from 'NAME' to 'tract_code'
    color_continuous_scale='Viridis',
    title='Social Vulnerability vs. Vegetation Index (Select points to highlight on map)',
    labels={
        'social_vulnerability_index': 'Social Vulnerability Index',
        'ndvi_mean': 'Mean Vegetation Index (NDVI)',
        'priority_score': 'Priority Score',
        'total_population': 'Population'
    }
)

scatter_fig.update_layout(
    dragmode='lasso',
    uirevision='constant',
    clickmode='event+select'
)

# Create the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Environmental Justice Analysis Dashboard"),
    html.P("Exploring the relationship between social vulnerability and environmental conditions"),

    # Main content area with map and scatter plot side by side
    html.Div([
        html.Div([
            dcc.Graph(id='svi-map', figure=svi_fig, style={'height': '50vw'})
        ], style={'width': '50%', 'padding': '0 5px', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='scatter-plot', figure=scatter_fig, style={'height': '50vw'})
        ], style={'width': '50%', 'padding': '0 5px', 'display': 'inline-block'})
    ], style={'display': 'flex', 'margin-bottom': '10px'}),

    # Table section below the plots
    html.Div([
        html.H3("Top 10 Priority Areas for Environmental Justice Interventions"),
        html.Table([
            html.Thead(
                html.Tr([
                    html.Th("Census Tract"),
                    html.Th("Social Vulnerability Index"),
                    html.Th("Vegetation Index"),
                    html.Th("Priority Score")
                ])
            ),
            html.Tbody([
                html.Tr([
                    html.Td(high_priority.iloc[i]['tract_code']),  # Changed from 'NAME'
                    html.Td(f"{high_priority.iloc[i]['social_vulnerability_index']:.3f}"),
                    html.Td(f"{high_priority.iloc[i]['ndvi_mean']:.3f}"),
                    html.Td(f"{high_priority.iloc[i]['priority_score']:.3f}")
                ]) for i in range(len(high_priority))
            ])
        ], style={'width': '100%', 'border': '1px solid black', 'margin-top': '20px'})
    ], style={'padding': '10px'})
])

# Debug callback - let's see what's actually happening
@app.callback(
    Output('svi-map', 'figure'),
    Input('scatter-plot', 'selectedData'),
    State('svi-map', 'figure'),
    prevent_initial_call=True
)
def update_svi_map_on_selection(selected_data, current_svi_fig):
    print("\n" + "="*50)
    print("CALLBACK TRIGGERED!")
    print("="*50)

    # Debug: Print the entire selected_data structure
    print(f"Selected data type: {type(selected_data)}")
    print(f"Selected data: {selected_data}")

    # Create base figure (same as original but preserve zoom/center if available)
    try:
        current_zoom = current_svi_fig['layout']['mapbox']['zoom']
        current_center = current_svi_fig['layout']['mapbox']['center']
        print(f"Preserving zoom: {current_zoom}, center: {current_center}")
    except:
        current_zoom = 9
        current_center = map_center
        print("Using default zoom and center")

    # Recreate the base map
    updated_fig = px.choropleth_mapbox(
        gdf,
        geojson=census_geojson,
        locations=gdf.index,
        color='social_vulnerability_index',
        color_continuous_scale='Viridis',
        range_color=[0, 1],
        mapbox_style='carto-positron',
        zoom=current_zoom,
        center=current_center,
        opacity=0.7,
        hover_name='tract_code',  # Changed from 'NAME'
        hover_data={
            'ndvi_mean': ':.3f',
            'social_vulnerability_index': ':.3f',
            'prop_poverty': ':.3f',  # Changed from 'poverty_rate'
            'prop_nonwhite': ':.3f',  # Changed from 'pct_nonwhite'
            'priority_score': ':.3f'
        },
        title='Social Vulnerability by Census Tract'
    )

    updated_fig.update_layout(uirevision='constant')

    # Process selection
    if selected_data and 'points' in selected_data and selected_data['points']:
        print(f"Processing {len(selected_data['points'])} selected points")

        selected_indices = []
        for i, point in enumerate(selected_data['points']):
            print(f"Point {i}: {point}")
            if 'pointIndex' in point:
                selected_indices.append(point['pointIndex'])

        print(f"Selected indices: {selected_indices}")

        if selected_indices:
            # Get selected geometries
            selected_gdf = gdf.iloc[selected_indices].copy()
            print(f"Selected {len(selected_gdf)} geometries")

            # Create highlight - try multiple approaches
            if not selected_gdf.empty:
                try:
                    # Method 1: Add as mapbox layer
                    highlight_geojson = selected_gdf.__geo_interface__

                    highlight_layer = {
                        "sourcetype": "geojson",
                        "source": highlight_geojson,
                        "type": "line",
                        "color": "red",
                        "line": {"width": 4},
                        "below": "traces"
                    }

                    updated_fig.add_layer(highlight_layer)
                    print("Added highlight layer via mapbox layers")

                except Exception as e:
                    print(f"Mapbox layer method failed: {e}")

                    # Method 2: Add as separate trace
                    try:
                        import plotly.graph_objects as go

                        # Extract coordinates for line plotting
                        for idx, row in selected_gdf.iterrows():
                            geom = row.geometry
                            if hasattr(geom, 'exterior'):
                                # Polygon
                                coords = list(geom.exterior.coords)
                                lons, lats = zip(*coords)

                                updated_fig.add_trace(go.Scattermapbox(
                                    lon=list(lons),
                                    lat=list(lats),
                                    mode='lines',
                                    line=dict(width=4, color='red'),
                                    showlegend=False,
                                    hoverinfo='skip'
                                ))
                        print("Added highlight via scatter traces")

                    except Exception as e2:
                        print(f"Scatter trace method also failed: {e2}")
    else:
        print("No points selected or invalid selection data")

    print("Returning updated figure")
    print("="*50)
    return updated_fig

# Function to start ngrok tunnel
def start_ngrok():
    port = 8050

    try:
        existing_tunnels = ngrok.get_tunnels()
        for tunnel in existing_tunnels:
            config_dict = tunnel.config if isinstance(tunnel.config, dict) else tunnel.config.__dict__ if hasattr(tunnel.config, '__dict__') else {}
            tunnel_addr = config_dict.get('addr')

            if tunnel.public_url.startswith("https") and tunnel_addr and str(tunnel_addr).endswith(f":{port}"):
                print(f"Existing HTTPS ngrok tunnel found at: {tunnel.public_url}")
                return

        print(f"Attempting to establish ngrok tunnel for port {port}...")
        http_tunnel = ngrok.connect(port, bind_tls=True)
        print(f"ngrok tunnel established at: {http_tunnel.public_url}")
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
        print("Ensure ngrok is installed, authenticated with your authtoken, and no conflicting process is using the port.")

# Run the server and ngrok
if __name__ == '__main__':
    ngrok_thread = threading.Thread(target=start_ngrok, daemon=True)
    ngrok_thread.start()
    time.sleep(5)

    app.run(debug=True, jupyter_mode='inline')

## Wrap Up Discussion

What are the main caveats you experienced today or in the past coding with LLMs?
Any other thoughts?

####<font color="#FF69B4">PROMPT: "What are the main caveats when using Gemini or any LLM to help you code?"</font>

## Some Resources and References

- [NASA Harmonized Landsat Sentinel (HLS) Project](https://hls.gsfc.nasa.gov/)
- [US Census Bureau API](https://www.census.gov/data/developers/data-sets.html)
- [EPA's Environmental Justice Screening Tool (EJScreen)](https://www.epa.gov/ejscreen)
- [NASA SEDAC - Socioeconomic Data and Applications Center](https://sedac.ciesin.columbia.edu/)
- [LP DAAC - Getting Started with Cloud-Native HLS Data in Python](https://lpdaac.usgs.gov/resources/e-learning/getting-started-cloud-native-hls-data-python/)
- [Census Data API User Guide](https://www.census.gov/data/developers/guidance/api-user-guide.html)